<a href="https://colab.research.google.com/github/alexsalman/CSE247/blob/main/3D_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####**3D Convolutional Neural Network**
######*I am using 3D Convolutional Neural Network to extract the temporal and spatial information which are merged slowly throughout the whole network.*

In [2]:
# required libraries
import os
import cv2
import math
import random
import numpy as np
import datetime as dt
from collections import deque
import matplotlib.pyplot as plt
from google.colab import drive
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout
from keras.layers import BatchNormalization, GlobalAveragePooling3D
from keras import regularizers
from PIL import Image
%matplotlib inline
print(tf.version.VERSION)
# https://bleedai.com/human-activity-recognition-using-tensorflow-cnn-lstm/

2.8.0


In [3]:
# set Numpy, Python, and Tensorflow seeds to get consistent results on every execution
seed_constant = 27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [4]:
# mount dataset from google drive
drive.mount('/content/gdrive/', force_remount=True)
gdrive_path = '/content/gdrive' + '/My Drive/247/'
os.chdir(gdrive_path)
os.getcwd()

Mounted at /content/gdrive/


'/content/gdrive/My Drive/247'

In [5]:
# frame dimention
IMAGE_HEIGHT, IMAGE_WIDTH = 128, 128
# frame number for each video (depth)
SEQUENCE_LENGTH = 16
# video dir path
DATASET_DIR = gdrive_path + 'Cropped_videos'
# labels of classes
CLASSES_LIST = ['hemostasis', 'inflammatory', 'proliferative', 'maturation']

In [6]:
# image cropping
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

In [7]:
# https://medium.com/analytics-vidhya/video-preprocessor-and-augmentation-for-deep-learning-tasks-12dd3fcce868
def load_video(path, resize=(128, 128)):
    video_reader = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = video_reader.read()
            if not ret:
                  break
            # frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            black_frame = frame
            frames.append(frame)
    finally:
        video_reader.release()
    return np.array(frames) / 255.0

In [8]:
def create_dataset(state):
    # Declared Empty Lists to store the features, labels and video file path values.
    features = []
    labels = []
    video_files_paths = []
    # Iterating through all the classes mentioned in the classes list
    for class_index, class_name in enumerate(CLASSES_LIST):
        # Display the name of the class whose data is being extracted.
        print(f'Extracting Data of Class: {class_name} {state}')
        # Get the list of video files present in the specific class name directory.
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        # Iterate through all the files present in the files list.
        for file_name in files_list:
            # Get the complete video path.
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)
            # create testing data
            if state == 'test':
                # get the mouse number
                mouse_number = int(video_file_path.split(' ')[2].split('_')[1].split('-')[1])
                # get the mouse side (L or R)
                mouse_side = video_file_path.split(' ')[2].split('_')[1].split('-')[2]
                if mouse_number == 4 and mouse_side == 'L':
                    frames = load_video(video_file_path)
                    features.append(frames)
                    labels.append(class_index)
                    video_files_paths.append(video_file_path)
            # create validation data
            elif state == 'valid':
                # get the mouse number
                mouse_number = int(video_file_path.split(' ')[2].split('_')[1].split('-')[1])
                # get the mouse side (L or R)
                mouse_side = video_file_path.split(' ')[2].split('_')[1].split('-')[2]
                if mouse_number == 4 and mouse_side == 'R':
                    frames = load_video(video_file_path)
                    features.append(frames)
                    labels.append(class_index)
                    video_files_paths.append(video_file_path)
            # create training data
            else:
                # get the mouse number
                mouse_number = int(video_file_path.split(' ')[2].split('_')[1].split('-')[1])
                if mouse_number != 4:
                    frames = load_video(video_file_path)
                    features.append(frames)
                    labels.append(class_index)
                    video_files_paths.append(video_file_path)
    # Converting the list to numpy arrays
    features = np.asarray(features)
    # print(features)
    labels = np.array(labels)
    # Return the frames, class index, and video file path.
    return features, labels, video_files_paths

In [9]:
# 6 mice for training, 2 mice for test and validation (one wound on each mice for test one for validation)
features_train, labels_train, video_files_paths_train = create_dataset('train')
features_test, labels_test, video_files_paths_test = create_dataset('test')
features_valid, labels_valid, video_files_paths_valid = create_dataset('valid')

Extracting Data of Class: hemostasis train
Extracting Data of Class: inflammatory train
Extracting Data of Class: proliferative train
Extracting Data of Class: maturation train
Extracting Data of Class: hemostasis test
Extracting Data of Class: inflammatory test
Extracting Data of Class: proliferative test
Extracting Data of Class: maturation test
Extracting Data of Class: hemostasis valid
Extracting Data of Class: inflammatory valid
Extracting Data of Class: proliferative valid
Extracting Data of Class: maturation valid


In [10]:
# one_hot_encoded_labels
labels_train = keras.utils.to_categorical(labels_train)
labels_test = keras.utils.to_categorical(labels_test)
labels_valid = keras.utils.to_categorical(labels_valid)

In [11]:
def create_3D_CNN_model():
    sample_shape = (16, 128, 128, 3)
    model = Sequential()

    model.add(Conv3D(32, 3, activation='relu', kernel_initializer='he_uniform',
                     kernel_regularizer=regularizers.L2(l2=1e-4),
                     input_shape=sample_shape))
    
    model.add(MaxPooling3D(2))

    model.add(Dropout(0.5))

    model.add(Conv3D(32, 3, activation='relu', kernel_initializer='he_uniform',
                     kernel_regularizer=regularizers.L2(l2=1e-4)))
    
    model.add(MaxPooling3D(2))

    model.add(Dropout(0.5))

    model.add(GlobalAveragePooling3D())

    model.add(Dense(32, activation='relu', kernel_initializer='he_uniform',
                    kernel_regularizer=regularizers.L2(l2=1e-4)))
    
    model.add(Dropout(0.5))

    model.add(Dense(16, activation='relu', kernel_initializer='he_uniform',
                    kernel_regularizer=regularizers.L2(l2=1e-4)))
    
    model.add(Dropout(0.6))

    model.add(Dense(8, activation='relu', kernel_initializer='he_uniform',
                    kernel_regularizer=regularizers.L2(l2=1e-4)))

    model.add(Dense(len(CLASSES_LIST), activation='softmax'))

    model.summary(line_length = 125)

    return model

In [12]:
# Construct the required convlstm model.
model = create_3D_CNN_model()
 
# Display the success message. 
print("Model Created Successfully!")

Model: "sequential"
_____________________________________________________________________________________________________________________________
 Layer (type)                                           Output Shape                                      Param #            
 conv3d (Conv3D)                                        (None, 14, 126, 126, 32)                          2624               
                                                                                                                             
 max_pooling3d (MaxPooling3D)                           (None, 7, 63, 63, 32)                             0                  
                                                                                                                             
 dropout (Dropout)                                      (None, 7, 63, 63, 32)                             0                  
                                                                                                  

In [13]:
# Create an Instance of Early Stopping Callback
early_stopping_callback = keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                                        patience = 20,
                                                        mode = 'min',
                                                        restore_best_weights = True)
# Compile the model and specify loss function, optimizer and metrics values to the model
model.compile(loss = 'categorical_crossentropy',
              optimizer= keras.optimizers.Adam(0.001, decay=1e-4),
              metrics=['accuracy'])
# Start training the model.
cnn_3d_model_training_history = model.fit(x = features_train,
                                          y = labels_train,
                                          epochs=100,
                                          batch_size=16,
                                          shuffle = True,
                                          validation_data = (features_valid, labels_valid))
                                          # callbacks = [early_stopping_callback])

Epoch 1/100
101/101 [==============================] - 21s 104ms/step - loss: 1.3244 - accuracy: 0.4208 - val_loss: 1.2701 - val_accuracy: 0.3860
Epoch 2/100
101/101 [==============================] - 10s 97ms/step - loss: 1.1651 - accuracy: 0.4146 - val_loss: 1.0822 - val_accuracy: 0.3566
Epoch 3/100
101/101 [==============================] - 10s 97ms/step - loss: 1.1015 - accuracy: 0.4270 - val_loss: 0.9988 - val_accuracy: 0.4338
Epoch 4/100
101/101 [==============================] - 10s 96ms/step - loss: 1.0428 - accuracy: 0.4759 - val_loss: 0.9496 - val_accuracy: 0.5110
Epoch 5/100
101/101 [==============================] - 10s 97ms/step - loss: 1.0017 - accuracy: 0.4808 - val_loss: 0.9077 - val_accuracy: 0.5846
Epoch 6/100
101/101 [==============================] - 10s 97ms/step - loss: 0.9701 - accuracy: 0.4963 - val_loss: 0.8976 - val_accuracy: 0.5662
Epoch 7/100
101/101 [==============================] - 10s 96ms/step - loss: 0.9490 - accuracy: 0.5186 - val_loss: 0.8552 - val_a

In [14]:
model_evaluation_history = model.evaluate(features_test, labels_test)
print('\n')
train_loss, train_acc = model.evaluate(features_train, labels_train, verbose=0)
test_loss, test_acc = model.evaluate(features_test, labels_test, verbose=0)
val_loss, val_acc = model.evaluate(features_valid, labels_valid, verbose=0)
print(f'Train accuracy: {train_acc*100:.3f} % || Test accuracy: {test_acc*100:.3f} % || Val accuracy: {val_acc*100:.3f} %')
print(f'Train loss: {train_loss:.3f} || Test loss: {test_loss:.3f} || Val loss: {val_loss:.3f}')

9/9 [==============================] - 1s 71ms/step - loss: 0.9013 - accuracy: 0.7353


Train accuracy: 86.757 % || Test accuracy: 73.529 % || Val accuracy: 69.485 %
Train loss: 0.347 || Test loss: 0.901 || Val loss: 0.614


In [ ]:
# Get the loss and accuracy from model_evaluation_history.
model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history
 
# Define the string date format.
# Get the current Date and Time in a DateTime Object.
# Convert the DateTime object to string according to the style mentioned in date_time_format string.
date_time_format = '%Y_%m_%d__%H_%M_%S'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)
 
# Define a useful name for our model to make it easy for us while navigating through multiple saved models.
model_file_name = f'3D_CNN_model___Date_Time_{current_date_time_string}___Loss_{model_evaluation_loss}___Accuracy_{model_evaluation_accuracy}.h5'
# Change dir
gdrive_path = '/content/gdrive' + '/My Drive/247/Saved_models/'
os.chdir(gdrive_path)
# Create a floder for the model files
!mkdir -p cnn_3d_{current_date_time_string}
# Save your Model.
model.save('convlstm_' + str(current_date_time_string) + '/' + model_file_name)
# Save model weights
model.save_weights('convlstm_' + str(current_date_time_string) + '/' + 'weights')

In [ ]:
def plot_metric(model_training_history, metric_name_1, metric_name_2, plot_name):
    '''
    This function will plot the metrics passed to it in a graph.
    Args:
        model_training_history: A history object containing a record of training and validation 
                                loss values and metrics values at successive epochs
        metric_name_1:          The name of the first metric that needs to be plotted in the graph.
        metric_name_2:          The name of the second metric that needs to be plotted in the graph.
        plot_name:              The title of the graph.
    '''
    
    # Get metric values using metric names as identifiers.
    metric_value_1 = model_training_history[0].history[metric_name_1]
    metric_value_2 = model_training_history[0].history[metric_name_2]
    
    # Construct a range object which will be used as x-axis (horizontal plane) of the graph.
    epochs = range(len(metric_value_1))

    # Plot the Graph.
    plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
    plt.plot(epochs, metric_value_2, 'red', label = metric_name_2)

    # Add title to the plot.
    plt.title(str(plot_name))

    # Add legend to the plot.
    plt.legend()

In [ ]:
plot_metric(cnn_3d_model_training_history, 'loss', 'val_loss', 'Total Loss vs Total Validation Loss')

In [ ]:
plot_metric(cnn_3d_model_training_history, 'accuracy', 'val_accuracy', 'Total Accuracy vs Total Validation Accuracy') 